# Metodo guidotti
L'idea è quella di trasformare ogni traiettoria in una sequenza di simboli, estrarre i motif e usarli per classificare

In [1]:
import time
from concurrent.futures import ThreadPoolExecutor
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import math
from datetime import timedelta, datetime
import plotly.express as px
from geolib import geohash
from sklearn.model_selection import train_test_split
from sklearn_extra.cluster import KMedoids


executor = ThreadPoolExecutor(max_workers=12)

df = pd.read_csv("../../examples/Animals Dataset/data/animals_preapred.zip").sort_values(by=["tid", 't'])

df.tid -= df.tid.min()

def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                results['mean_test_score'][candidate],
                results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

df.head()

,tid,class,t,c1,c2
0,0,D,0,50.1066,3.79665
1,0,D,4,50.1045,3.79455
2,0,D,7,50.1111,3.79845
3,0,D,9,50.1072,3.79845
4,0,D,15,50.1132,3.79965


In [35]:
t = 0

In [36]:
%%time

start = datetime.now()

precision =  4 #da 1 a 12

df["geohash"] = [geohash.encode(x[0], x[1], precision) for x in tqdm(df[["c1", "c2"]].values)]

t += (datetime.now()-start).total_seconds()*1000

  0%|          | 0/14990 [00:00<?, ?it/s]

Wall time: 1.1 s


In [37]:
t

1103.001

In [38]:
%%time

start = datetime.now()

decodes = dict()

for gh in tqdm(df.geohash.unique()):
    decodes[gh] = geohash.bounds(gh)

l = []
prec = "!"
c = -1
for gh in df.geohash:
    if gh != prec:
        prec = gh
        c += 1
    l.append(gh+"_"+str(c))

df["geohash2"] = l

df["lat_norm"] = df.c1 - df.geohash.transform(lambda x: decodes[x].sw.lat)
df["lon_norm"] = df.c2 - df.geohash.transform(lambda x: decodes[x].sw.lon)

t += (datetime.now()-start).total_seconds()*1000

  0%|          | 0/2 [00:00<?, ?it/s]

Wall time: 256 ms


In [39]:
px.line_3d(df[df.tid == 1], x="t", y="c1", z="c2", color="geohash2")

In [40]:
px.line_3d(df[df.tid == 1], x="t", y="lat_norm", z="lon_norm", color="geohash2")

In [41]:
px.line(df[df.tid==1], x="lat_norm", y="lon_norm", color="geohash2")

In [42]:
px.line_3d(df[df.tid == 1], x="t", y="c1", z="c2")

In [43]:
%%time

start = datetime.now()

tid_train, tid_test, _, _ = train_test_split(df.groupby(by=["tid"]).max().reset_index()["tid"],
                                                        df.groupby(by=["tid"]).max().reset_index()["class"],
                                                        test_size=.3,
                                                        stratify=df.groupby(by=["tid"]).max().reset_index()["class"],
                                                        random_state=3)

df_train = df[df.tid.isin(tid_train)]
df_test = df[~df.tid.isin(tid_train)]

t+= (datetime.now()-start).total_seconds()*1000
df_train

Wall time: 475 ms


,tid,class,t,c1,c2,geohash,geohash2,lat_norm,lon_norm
0,0,D,0,50.1066,3.79665,u0ft,u0ft_0,0.008944,0.281025
1,0,D,4,50.1045,3.79455,u0ft,u0ft_0,0.006844,0.278925
2,0,D,7,50.1111,3.79845,u0ft,u0ft_0,0.013444,0.282825
3,0,D,9,50.1072,3.79845,u0ft,u0ft_0,0.009544,0.282825
4,0,D,15,50.1132,3.79965,u0ft,u0ft_0,0.015544,0.284025
...,...,...,...,...,...,...,...,...,...
14863,100,C,260,50.1768,3.76185,u0ft,u0ft_486,0.079144,0.246225
14864,100,C,262,50.1765,3.76095,u0ft,u0ft_486,0.078844,0.245325
14865,100,C,264,50.1753,3.76005,u0ft,u0ft_486,0.077644,0.244425
14866,100,C,266,50.1717,3.76545,u0ft,u0ft_486,0.074044,0.249825


In [44]:
%%time

start = datetime.now()

movelets = []

movelet = []
movelets.append(movelet)
prec_gh2 = df_train.values[0, 6]
first_row = df_train.values[0]
for row in tqdm(df_train.values):
    if row[6] != prec_gh2:
        prec_gh2 = row[6]
        first_row = row
        movelet = []
        movelets.append(movelet)

    movelet.append((row[-2]-first_row[-2], row[-1]-first_row[-1], row[2]-first_row[2]))

t+= (datetime.now()-start).total_seconds()*1000

  0%|          | 0/10600 [00:00<?, ?it/s]

Wall time: 150 ms


In [45]:
#From here credits to Riccardo Guidotti

def spherical_distance(a, b):
    lat1 = a[1]
    lon1 = a[0]
    lat2 = b[1]
    lon2 = b[0]
    R = 6371000
    rlon1 = lon1 * math.pi / 180.0
    rlon2 = lon2 * math.pi / 180.0
    rlat1 = lat1 * math.pi / 180.0
    rlat2 = lat2 * math.pi / 180.0
    dlon = (rlon1 - rlon2) / 2.0
    dlat = (rlat1 - rlat2) / 2.0
    lat12 = (rlat1 + rlat2) / 2.0
    sindlat = math.sin(dlat)
    sindlon = math.sin(dlon)
    cosdlon = math.cos(dlon)
    coslat12 = math.cos(lat12)
    f = sindlat * sindlat * cosdlon * cosdlon + sindlon * sindlon * coslat12 * coslat12
    f = math.sqrt(f)
    f = math.asin(f) * 2.0 # the angle between the points
    f *= R
    return f

def trajectory_distance(tr1, tr2):
    if len(tr1) == 1 and len(tr2) == 1:
        return .0

    i1 = 0
    i2 = 0
    np = 0

    last_tr1 = tr1[i1]
    last_tr2 = tr2[i2]

    dist = spherical_distance(last_tr1, last_tr2)
    np += 1

    while True:
        if i1 >= (len(tr1)-1) or i2 >= (len(tr2)-1):
            break

        step_tr1 = spherical_distance(last_tr1, tr1[i1 + 1])
        step_tr2 = spherical_distance(last_tr2, tr2[i2 + 1])

        if step_tr1 < step_tr2:
            i1 += 1
            last_tr1 = tr1[i1]
            last_tr2 = closest_point_on_segment(last_tr2, tr2[i2+1], last_tr1)
        elif step_tr1 > step_tr2:
            i2 += 1
            last_tr2 = tr2[i2]
            last_tr1 = closest_point_on_segment(last_tr1, tr1[i1+1], last_tr2)
        else:
            i1 += 1
            i2 += 1
            last_tr1 = tr1[i1]
            last_tr2 = tr2[i2]

        d = spherical_distance(last_tr1, last_tr2)

        dist += d
        np += 1

    for i in range(i1, len(tr1)):
        d = spherical_distance(tr2[-1], tr1[i])
        dist += d
        np += 1

    for i in range(i2, len(tr2)):
        d = spherical_distance(tr1[-1], tr2[i])
        dist += d
        np += 1

    dist = 1.0 * dist / np

    return dist

def closest_point_on_segment(a, b, p):
    sx1 = a[0]
    sx2 = b[0]
    sy1 = a[1]
    sy2 = b[1]
    sz1 = a[2]
    sz2 = b[2]
    px = p[0]
    py = p[1]

    x_delta = sx2 - sx1
    y_delta = sy2 - sy1
    z_delta = sz2 - sz1

    if x_delta == 0 and y_delta == 0:
        return p

    u = ((px - sx1) * x_delta + (py - sy1) * y_delta) / (x_delta * x_delta + y_delta * y_delta)
    if u < 0:
        closest_point = a
    elif u > 1:
        closest_point = b
    else:
        cp_x = sx1 + u * x_delta
        cp_y = sy1 + u * y_delta
        dist_a_cp = spherical_distance(a, [cp_x, cp_y, 0])
        if dist_a_cp != 0:
            cp_z = sz1 + z_delta / (spherical_distance(a, b) / spherical_distance(a, [cp_x, cp_y, 0]))
        else:
            cp_z = a[2]
        closest_point = [cp_x, cp_y, cp_z]

    return closest_point

In [46]:
%%time

start = datetime.now()

colToDelete = 0

new_movelets = []

for i in range(len(movelets)):
    if(len(movelets[i]) < 5):
        colToDelete += 1
        continue

    new_movelets.append(movelets[i])

t+= (datetime.now()-start).total_seconds()*1000

Wall time: 1 ms


In [47]:
%%time

start = datetime.now()

dim = len(new_movelets)

dist_matrix = np.zeros((dim, dim))

for i, tr1 in enumerate(tqdm(new_movelets)):
    for j, tr2 in enumerate(new_movelets[:i+1]):
        dist = trajectory_distance(tr1, tr2)
        dist_matrix[i, j] = dist
        dist_matrix[j, i] = dist
        
t+= (datetime.now()-start).total_seconds()*1000

  0%|          | 0/179 [00:00<?, ?it/s]

Wall time: 15.5 s


In [48]:
len(movelets[0])

345

In [49]:
t

17438.003

# QUI

In [50]:
t = 0

def symbolize(medoids, tr):
    i = 0
    sym = []
    while i < len(tr):
        best_medoid_id = np.inf
        best_medoid_score = np.inf
        for j, medoid in enumerate(medoids):
            score = trajectory_distance(medoid, list(map(lambda x: (x[0]-tr[i][0], x[1]-tr[i][1], x[2]-tr[i][2]), tr[i:i+len(medoid)])))
            if best_medoid_score > score:
                best_medoid_id = j
                best_medoid_score = score
        sym.append(best_medoid_id)
        i += len(medoids[best_medoid_id])

    return sym

def myContains(big, small):
    count = 0
    for i in range(len(big)):
        if big[i] == small[count]:
            count +=1
        else:
            count = 0
        if count == len(small):
            return True

    return False

In [51]:
from sklearn.feature_selection import mutual_info_classif

y=df.groupby(by="tid").max()["class"]

In [52]:
def guidotti(n_clust=20, sliding_w=3, top_k=200):
    kmedoids = KMedoids(n_clusters=n_clust, random_state=0, metric="precomputed").fit(dist_matrix)    
    medoids = [new_movelets[x] for x in kmedoids.medoid_indices_]
    trajectories = [[(x[3], x[4], x[2]) for x in df[df.tid == tid].values] for tid in df.tid.unique()]
    
    trajectories_sym=[]
    for trajectory in tqdm(trajectories, desc="symbolize"):
        trajectories_sym.append(symbolize(medoids, trajectory))
            
    trajectories_sym_train, _, y_sym_tr, _ = train_test_split(trajectories_sym,
                                                        y,
                                                        test_size=.3,
                                                        stratify=y,
                                                        random_state=3)
    
    subtrajectories_sym=[]
    for i, tr in enumerate(tqdm(trajectories_sym_train, desc="sliding w")):
        for j in range(0, len(tr)-sliding_w):
            subtrajectories_sym.append(tr[j:j+sliding_w])
    
    sliding_w *= 2
    for i, tr in enumerate(tqdm(trajectories_sym_train, desc="sliding w")):
        for j in range(0, len(tr)-sliding_w):
            subtrajectories_sym.append(tr[j:j+sliding_w])
    
    sliding_w *= 2
    for i, tr in enumerate(tqdm(trajectories_sym_train, desc="sliding w")):
        for j in range(0, len(tr)-sliding_w):
            subtrajectories_sym.append(tr[j:j+sliding_w])
    
    matr_sub = np.ones((len(trajectories_sym_train), len(subtrajectories_sym)))*np.NAN
    for i, traiettoria in enumerate(tqdm(trajectories_sym_train, desc="contains")):
        for j, pattern in enumerate(subtrajectories_sym):
            matr_sub[i][j] = 1 if myContains(traiettoria, pattern) else 0
    
    mi = mutual_info_classif(matr_sub, y_sym_tr)
    d = {i: x for i, x in enumerate(mi)}
    d = {k: v for k, v in sorted(d.items(), key=lambda item: -item[1])}
    
    res = []
    for key in list(d.keys())[:top_k]:
        res.append(subtrajectories_sym[key])
        
    matr = np.ones((len(trajectories_sym), len(res)))*np.NAN
    for i, traiettoria in enumerate(tqdm(trajectories_sym)):
        for j, pattern in enumerate(res):
            matr[i][j] = 1 if myContains(traiettoria, pattern) else 0
    
    return matr

In [53]:
def runTest(n_clust, sliding_w, top_k):
    start = datetime.now()
    
    X = guidotti(n_clust=n_clust, sliding_w=sliding_w, top_k=top_k)
    
    return (datetime.now()-start).total_seconds()*1000, X

In [54]:
def foo(x):
    return (x, x*x)

In [55]:
from concurrent.futures import ProcessPoolExecutor

executor = ProcessPoolExecutor(max_workers=1)

n_clusters = [2, 5, 20, 100]
sliding_windows = [2, 3, 5]
top_k = [1, 2, 10, 50]

res_t = []
res_X = []

conf = []
processes = []
for n_clust in tqdm(n_clusters, position=0):
    for sliding_w in sliding_windows:
        for k in top_k:
            try:
                conf.append((n_clust, sliding_w, k))
                #processes.append(executor.submit(foo, 1))
                #processes.append(executor.submit(runTest, n_clust, sliding_w, k))
                t, x = runTest(n_clust, sliding_w, k)
                res_t.append(t)
                res_X.append(x)
            except:
                print("FAIL")
                res_t.append(np.NAN)
                res_X.append(np.NAN)


  0%|          | 0/4 [00:00<?, ?it/s]

symbolize:   0%|          | 0/102 [00:00<?, ?it/s]

sliding w:   0%|          | 0/71 [00:00<?, ?it/s]

sliding w:   0%|          | 0/71 [00:00<?, ?it/s]

sliding w:   0%|          | 0/71 [00:00<?, ?it/s]

...

sliding w:   0%|          | 0/71 [00:00<?, ?it/s]

sliding w:   0%|          | 0/71 [00:00<?, ?it/s]

sliding w:   0%|          | 0/71 [00:00<?, ?it/s]

contains:   0%|          | 0/71 [00:00<?, ?it/s]

  0%|          | 0/102 [00:00<?, ?it/s]

In [56]:
conf

[(2, 2, 1),
 (2, 2, 2),
 (2, 2, 10),
 (2, 2, 50),
 (2, 3, 1),
 (2, 3, 2),
 (2, 3, 10),
 (2, 3, 50),
 (2, 5, 1),
 (2, 5, 2),
 (2, 5, 10),
 (2, 5, 50),
 (5, 2, 1),
 (5, 2, 2),
 (5, 2, 10),
 (5, 2, 50),
 (5, 3, 1),
 (5, 3, 2),
 (5, 3, 10),
 (5, 3, 50),
 (5, 5, 1),
 (5, 5, 2),
 (5, 5, 10),
 (5, 5, 50),
 (20, 2, 1),
 (20, 2, 2),
 (20, 2, 10),
 (20, 2, 50),
 (20, 3, 1),
 (20, 3, 2),
 (20, 3, 10),
 (20, 3, 50),
 (20, 5, 1),
 (20, 5, 2),
 (20, 5, 10),
 (20, 5, 50),
 (100, 2, 1),
 (100, 2, 2),
 (100, 2, 10),
 (100, 2, 50),
 (100, 3, 1),
 (100, 3, 2),
 (100, 3, 10),
 (100, 3, 50),
 (100, 5, 1),
 (100, 5, 2),
 (100, 5, 10),
 (100, 5, 50)]

In [57]:
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score

def compute_measures(test, pred):
    return (accuracy_score(test, pred), precision_score(test, pred, average="micro"), f1_score(test, pred, average="micro"), recall_score(test, pred, average="micro"))

res_val = []

for x in tqdm(res_X):
    try:

        X = x
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, stratify=y, random_state=3)

        tree_para = {'n_estimators':range(100, 1500, 300),
                     'max_depth':range(2, 18, 3)}
        gs = GridSearchCV(RandomForestClassifier(), tree_para, cv=5, n_jobs=12, verbose=0)
        result = gs.fit(X_train, y_train)

        y_pred = result.best_estimator_.fit(X_train, y_train).predict(X_test)

        res_val.append(compute_measures(y_test, y_pred))

        print(compute_measures(y_test, y_pred))
    except:
        print("skip")
        res_val.append((np.NAN, np.NAN, np.NAN, np.NAN))

  0%|          | 0/48 [00:00<?, ?it/s]

(0.41935483870967744, 0.41935483870967744, 0.41935483870967744, 0.41935483870967744)
(0.41935483870967744, 0.41935483870967744, 0.41935483870967744, 0.41935483870967744)
(0.3870967741935484, 0.3870967741935484, 0.3870967741935484, 0.3870967741935484)
(0.45161290322580644, 0.45161290322580644, 0.45161290322580644, 0.45161290322580644)
(0.45161290322580644, 0.45161290322580644, 0.45161290322580644, 0.45161290322580644)
(0.45161290322580644, 0.45161290322580644, 0.45161290322580644, 0.45161290322580644)
(0.5483870967741935, 0.5483870967741935, 0.5483870967741935, 0.5483870967741935)
(0.5483870967741935, 0.5483870967741935, 0.5483870967741935, 0.5483870967741935)
(0.3548387096774194, 0.3548387096774194, 0.35483870967741943, 0.3548387096774194)
(0.5161290322580645, 0.5161290322580645, 0.5161290322580645, 0.5161290322580645)
(0.5483870967741935, 0.5483870967741935, 0.5483870967741935, 0.5483870967741935)
(0.5161290322580645, 0.5161290322580645, 0.5161290322580645, 0.5161290322580645)
(0.5806

In [58]:
df_res = pd.DataFrame(res_val, columns=["acc", "precision", "f1", "recall"])

df_res[["nclust", "sliding", "k"]] = conf

df_res["t"] = res_t

df_res["t"] += 17438

df_res.to_csv("Animals search multiple rappresentation precision 4.csv", index=None)

df_res

,acc,precision,f1,recall,nclust,sliding,k,t
0,0.419355,0.419355,0.419355,0.419355,2,2,1,22502.998
1,0.419355,0.419355,0.419355,0.419355,2,2,2,22123.002
2,0.387097,0.387097,0.387097,0.387097,2,2,10,22176.000
3,0.451613,0.451613,0.451613,0.451613,2,2,50,23187.687
4,0.451613,0.451613,0.451613,0.451613,2,3,1,21093.001
5,0.451613,0.451613,0.451613,0.451613,2,3,2,19479.998
6,0.548387,0.548387,0.548387,0.548387,2,3,10,21431.999
7,0.548387,0.548387,0.548387,0.548387,2,3,50,21620.003
8,0.354839,0.354839,0.354839,0.354839,2,5,1,20141.997
9,0.516129,0.516129,0.516129,0.516129,2,5,2,19832.003


In [59]:
df_res.sort_values(by="acc", ascending=False).head()

,acc,precision,f1,recall,nclust,sliding,k,t
17,0.741935,0.741935,0.741935,0.741935,5,3,2,25409.998
18,0.645161,0.645161,0.645161,0.645161,5,3,10,28164.001
13,0.580645,0.580645,0.580645,0.580645,5,2,2,30604.000
14,0.580645,0.580645,0.580645,0.580645,5,2,10,27725.999
12,0.580645,0.580645,0.580645,0.580645,5,2,1,27884.000


In [27]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

tree_para = {'n_estimators':range(100, 1500, 300),
             'max_depth':range(2, 18, 3)}
gs = GridSearchCV(RandomForestClassifier(), tree_para, cv=10, n_jobs=12, verbose=3)
result = gs.fit(X_train, y_train)

report(gs.cv_results_, n_top=1)

Fitting 10 folds for each of 30 candidates, totalling 300 fits
Model with rank: 1
Mean validation score: 0.337 (std: 0.065)
Parameters: {'max_depth': 5, 'n_estimators': 400}

Model with rank: 1
Mean validation score: 0.337 (std: 0.065)
Parameters: {'max_depth': 5, 'n_estimators': 1000}

Model with rank: 1
Mean validation score: 0.337 (std: 0.065)
Parameters: {'max_depth': 5, 'n_estimators': 1300}

Model with rank: 1
Mean validation score: 0.337 (std: 0.065)
Parameters: {'max_depth': 8, 'n_estimators': 100}

Model with rank: 1
Mean validation score: 0.337 (std: 0.065)
Parameters: {'max_depth': 8, 'n_estimators': 400}

Model with rank: 1
Mean validation score: 0.337 (std: 0.065)
Parameters: {'max_depth': 8, 'n_estimators': 700}

Model with rank: 1
Mean validation score: 0.337 (std: 0.065)
Parameters: {'max_depth': 8, 'n_estimators': 1000}

Model with rank: 1
Mean validation score: 0.337 (std: 0.065)
Parameters: {'max_depth': 8, 'n_estimators': 1300}

Model with rank: 1
Mean validation sc

In [28]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=2, random_state=3, n_jobs=-1, n_estimators=400)

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           C       0.00      0.00      0.00        10
           D       0.00      0.00      0.00         9
           E       0.39      1.00      0.56        12

    accuracy                           0.39        31
   macro avg       0.13      0.33      0.19        31
weighted avg       0.15      0.39      0.22        31



c:\users\cri98\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

c:\users\cri98\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

c:\users\cri98\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [ ]:
print(classification_report(y_test, result.best_estimator_.fit(X_train, y_train).predict(X_test)))